<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [30]:
# TODO - Words, words, mere words, no matter from the heart.

### Important Imports

In [2]:
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import requests
import numpy as np
import random
import sys
import os

max_features = 20000
maxlen = 80
batch_size = 32

## Creating streamlined function for text processing

In [3]:
# Setting up function for streamlined processing
def process_text(text_file):
    """Opens and processes text to be ready for model fitting"""

    # Import and encoding
    response = requests.get(text_file)
    train_text = response.text
    response.encoding = 'utf-8'
    # re.sub('[^A-Za-z0-9]+', '', train_text)
    len(train_text) # For debugging
    print(train_text[:200]) # For debugging

    # Encoding data as chars
    chars = list(set(train_text))
    char_int = {c:i for i, c in enumerate(chars)}
    int_char = {i:c for i, c in enumerate(chars)}

    # Create the Sequence Data
    maxlen = 40
    step = 5
    encoded = [char_int[c] for c in train_text]
    sequences = [] # Each element is 40 characters long
    next_chars = [] # One element for each sequence
    for i in range(0, len(encoded) - maxlen, step):
        sequences.append(encoded[i : i + maxlen])
        next_chars.append(encoded[i + maxlen])
    # print('sequences:', len(sequences)) # For debugging

    # Specify x & y
    x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
    for i, sequence in enumerate(sequences):
        for t, char in enumerate(sequence):
            x[i, t, char] = 1
        y[i, next_chars[i]] = 1
    # print(x.shape, y.shape) # For debugging
    return x, y, train_text

def on_epoch_end(epoch, _):
    """ Prints generated text one char at a time for each epoch on trained data"""
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(train_text) - maxlen - 1)

    for diversity in [0.2, 0.5, 0.7, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = train_text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]
            # print("next_char: " + next_char)

            sentence = sentence[1:] + next_char
            # print("sentence[1:]: " + sentence[1:])

            sys.stdout.write(next_char)
            # sys.stdout.flush() # This is bugging, not adding any spaces
        print()

def fit_model(x, y, num=5):
    """Instantiates and fits the model"""
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, len(chars))))
    model.add(Dense(len(chars), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')  
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(x, y, batch_size=128, epochs=num, callbacks=[print_callback])

In [7]:
plays_sonnets = 'https://raw.githubusercontent.com/lechemrc/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/shakespeare_full.txt'

In [8]:
x, y = process_text(plays_sonnets)

﻿THE SONNETS

                    1

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as the riper should by time decease,
His tender heir might bear his memo


ValueError: too many values to unpack (expected 2)

In [ ]:
# fit_model(x, y) # This is not working, really not sure why...

In [143]:
on_epoch_end(5, _)


----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "           1192
  Holding their course "
           1192
  Holding their course

KeyError: '\r'

# Generating text from all works

In [15]:
# Import and encoding
response = requests.get(plays_sonnets)
train_text = response.text
response.encoding = 'utf-8'
print(len(train_text)) # For debugging
print(train_text[:200]) # For debugging

# Encoding data as chars
chars = list(set(train_text))
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

# Create the Sequence Data
maxlen = 40
step = 5
encoded = [char_int[c] for c in train_text]
sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence
for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
print('sequences:', len(sequences)) # For debugging

# Specify x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
    y[i, next_chars[i]] = 1

x.shape, y.shape 

5549280
﻿THE SONNETS

                    1

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as the riper should by time decease,
His tender heir might bear his memo
sequences: 1109848


((1109848, 40, 102), (1109848, 102))

In [16]:
def on_epoch_end(epoch, _):
    """ Prints generated text one char at a time for each epoch on trained data"""
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(train_text) - maxlen - 1)

    for diversity in [0.2, 0.5, 0.7, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = train_text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]
            # print("next_char: " + next_char)

            sentence = sentence[1:] + next_char
            # print("sentence[1:]: " + sentence[1:])

            sys.stdout.write(next_char)
            # sys.stdout.flush() # This is bugging, not adding any spaces
        print()

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')  
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y, batch_size=128, epochs=10, callbacks=[print_callback])

Train on 1109848 samples
Epoch 1/10
 915456/1109848 [=======================>......] - ETA: 1:12 - loss: 2.2946

## Generating text from Sonnets

In [129]:
sonnets = 'https://raw.githubusercontent.com/lechemrc/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/shakespeare_sonnets.txt'

In [130]:
# Import and encoding
response = requests.get(sonnets)
train_text = response.text
response.encoding = 'utf-8'
print(len(text)) # For debugging
print(train_text[:200]) # For debugging

# Encoding data as chars
chars = list(set(train_text))
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

# Create the Sequence Data
maxlen = 40
step = 5
encoded = [char_int[c] for c in train_text]
sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence
for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
print('sequences:', len(sequences)) # For debugging

# Specify x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
    y[i, next_chars[i]] = 1

x.shape, y.shape 

5777367
﻿THE SONNETS

                    1

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as the riper should by time decease,
His tender heir might bear his memo
sequences: 19663


((19663, 40, 73), (19663, 73))

In [132]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')  
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

Train on 19663 samples
19584/19663 [============================>.] - ETA: 0s - loss: 3.1593
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "dulness,
And give it way. I know thou c"
dulness,
And give it way. I know thou c

KeyError: '\r'

## Generating text from plays

In [ ]:
plays = 'https://raw.githubusercontent.com/lechemrc/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/shakespeare_plays.txt'


## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN

In [0]:
# Import and encoding
response = requests.get(full_text)
train_text = response.text
response.encoding = 'utf-8'
# len(text) # For debugging

# Encoding data as chars
chars = list(set(train_text))
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

# Create the Sequence Data
maxlen = 40
step = 5
encoded = [char_int[c] for c in train_text]
sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence
for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
# print('sequences:', len(sequences)) # For debugging

# Specify x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
    y[i, next_chars[i]] = 1
print(x.shape, y.shape) # For debugging
y